### Pipeline Stage 5: Matchy-9000

Welcome to Pipeline 5! In this stage, we will be using the data from Pipeline 3 to generate our final matches. The process involves leveraging the OpenAI API to generate a similarity score between job profiles and student profiles. By providing the API with both a job profile and a student profile, it returns a similarity score that helps us identify the best matches. This stage refines the matching process, ensuring that the most compatible candidates are paired with the most suitable job opportunities.

<p>Below the Matchy-9000 code, you will find previously generated results that you can review without needing to run Matchy-9000.<p>

In [1]:
#~~~
import re
from concurrent.futures import ThreadPoolExecutor, as_completed, wait
import concurrent.futures
import os 
import time 
import math
import glob
import pandas as pd
from IPython import get_ipython
from IPython.display import clear_output, display
import ast
from mypackage.utils import *
from tqdm.notebook import tqdm
from openai import OpenAI, RateLimitError, AzureOpenAI
import threading
import tiktoken
import ipywidgets as widgets
from statistics import mean
from collections import defaultdict


# Error handling
class RateLimitError(Exception):
    pass
class TimeoutException(Exception):
    pass

<h3>API Key Setup</h3>

<p>Below, you need to add your own API key. You can access these via Azure or OpenAI. Please note that they do cost money to use.</p>


In [2]:

# # OpenAI API key setup
# client = AzureOpenAI(
#     azure_endpoint = 'endpoint_here',
#     api_key= 'api_key_here',
#     api_version="2024-05-01-preview"
# )


# My API key
client = OpenAI(api_key='Your API key here')

In [ ]:
# Load the project path
path_to_project = load_project_path()

if path_to_project:
    print(f"Project path loaded: {path_to_project}")
else:
    print("Please set the project path in the initial notebook.")
    
SP_path = f'{path_to_project}/data/SP_table/SP4_post_keyword_filtering.parquet' 
SP = pd.read_parquet(SP_path)
print(SP.shape)

### AI Instructions

These are the instructions currently being provided to the API. They were drafted with the intent to guide the API in evaluating the alignment between internship applicants and specific job descriptions. However, these instructions should undergo an iterative process of improvement to enhance their effectiveness.

#### Key Features and Considerations

1. **Referencing**: 
   - One of the critical elements of these instructions is the emphasis on referencing. By including explicit instructions on how to reference the information, we can minimize hallucinations and ensure that the AI generates responses that are grounded in the provided data. 
   - The use of references, such as line numbers from both student profiles and job descriptions, helps maintain traceability and transparency in the AI's responses.

2. **Numbering Lines**:
   - I have found that numbering each line of the information provided to the AI is an effective method for enabling accurate referencing. This approach ensures that the AI can precisely identify and relate specific details from the input data to the task at hand.
   - Currently, the text being sent to the AI does not include numbered lines. It would be beneficial to refine the function that prepares the text to incorporate line numbering. This refinement would further enhance the accuracy of the AI's referencing and reduce the risk of errors.

#### Next Steps

- **Iterative Improvement**: The instructions should be regularly reviewed and updated based on feedback and results. This will help identify areas where the AI's performance can be enhanced and ensure that the instructions remain aligned with the project's goals.
- **Implementing Line Numbering**: Refining the function that prepares the text to include line numbers is a recommended next step. This will improve the AI's ability to reference information accurately and reduce the potential for generating content that is not directly tied to the provided data.

By following these recommendations, the instructions can be made more robust, leading to more accurate and reliable outcomes from the AI.

In [5]:
instruction = """
 Task:

Your task is to see how well internship applicants align with a specific job. I will be sending you different student profiles (which include essays, resumes, and other information about the student), and job descriptions. You will compute an alignment score between the student profile and the job description. The slignment will be broken down into four categories:

1. Company and Industry Alignment
2. Job Role and Responsibilities vs. Applicant Experience
3. Education, Technical Skills, and Tools
4. Values, perks, development opportunities, and Company Culture Alignment:


Alignment Score Notes:
- You will also calculate the overall similarity score by averaging the alignment scores from each category. 

Referencing Notes:
- When analyzing the alignment between the job description and the student profile, reference the line numbers from which the information is taken.
- Use the following format for referencing:
  - For information from the student profile, label it as (SP line #).
  - For information from the job description, label it as (JD line #).

Examples of referencing:
1. If a student profile on line 5 states, "5. I am good with SQL," and the job description on line 17 mentions, "17. We need someone who knows SQL":
   - *Output:* "The student's knowledge of SQL (SP line 5) aligns well with the job's requirement for someone skilled in this area (JD line 17)."
2. If a student profile expresses a strong preference for working in the beauty industry, saying on line 3, "3. I really only want to work in the beauty industry.", and the job description indicates the job is in the accounting industry on line 19 by stating, "19. This job is in the accounting industry.":
   - *Output:* "The student's desire to work in the beauty industry (SP line 3) does not align with the job's focus on the accounting industry (JD line 19)."

Additional Notes:
- It is EXTREAMLY important to adhead to the format provided under 'Expected Output:'.
- Do not include any additional explanations or words in the output beyond what is outlines under 'Expected Output:'
- All outputs should be in plain text without using any special formatting, such as HTML, Markdown, LaTeX, or any other formatting tags. Your output should be in plain text, following the formatting outlined under 'Expected Output:'. 
- Do not use any bold text (i.e. bold) or formatted headers. 


Expected Output:

Student Name: [Student Name Goes Here]

Category Breakdown

Company and Industry Alignment:
- Job Description: [Briefly describe the industry focus and mission of the company.]
- Applicant: [Briefly describe the applicant's industry interests and aspirations.
- Summary of Alignment: Provide a concise summary of how well the applicant's interests align with the company's industry and mission.
- Alignment Score: X.0/10

2. Job Role and Responsibilities vs. Applicant Experience:
- Job Description: [Outline the key responsibilities and roles in the job description.]
- Applicant: [Summarize the applicant's relevant experience and past roles.]
- Summary of Alignment: [Provide a summary of how well the applicant's experience matches the job responsibilities.]
- Alignment Score: X.0/10
[
3. Education, Skills, and Tools:
- Job Description: [List the required and prefered skills and tools needed for the job.]
- Applicant: [Highlight the applicant's education and and skill set.]
- Summary of Alignment: [Provide a summary of how well the applicant's skills match the job requirements.]
- Alignment Score: X.0/10

4. Values, perks, development opportunities, and Company Culture Alignment:
- Job Description: [Highlight the company's values, perks, culture, and development opportunities.]
- Applicant: [Describe the applicant's values and what they seek in company culture and benifits.]
- Summary of Alignment: [Provide a summary of how well the applicant aligns with the company.]
- Alignment Score: X.0/10

Overall Alignment Score:
[Overall Alignment Score Calculations Go Here] =  X.X/10

Conclusion:

[Provide a brief conclusion summarizing the overall alignment between the applicant and the job, highlighting key strengths and areas where alignment is lacking. This should help gauge whether the applicant would be a good fit for the role.]

Alignment Scores Summary:

Company and Industry Alignment Score: X.0/10
Job Role and Responsibilities vs. Applicant Experience Score: X.0/10
Education, Technical Skills, and Tools: X.0/10
Values, perks, development opportunities, and Company Culture Alignment: X.0/10
Overall Alignment Score: X.X/10
"""

## Matchy-9000: Automated Student-Job Profile Alignment

This Python script is designed to process and align student profiles with job descriptions using an AI-based scoring mechanism. The alignment process is broken down into several stages, including token estimation, batch processing, and result aggregation. The following description outlines the key components and functions used in this script.

### 1. **Loading and Combining DataFrames**
   - **Function:** `load_and_combine_saved_dfs(timeout_dir, base_filename)`
   - **Description:** This function loads all previously saved DataFrames from a specified directory and combines them into a single DataFrame. It is useful for recovering progress from temporary files in case of a timeout or interruption.

### 2. **Restarting the Processing**
   - **Function:** `restart_processing(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch)`
   - **Description:** This function restarts the processing of profiles by loading previously saved progress and marking already processed rows. It is useful for resuming work after an interruption.

### 3. **Saving Temporary DataFrame**
   - **Function:** `save_temp_dataframe(dataframe, timeout_dir, base_filename)`
   - **Description:** This function saves the processed rows of the DataFrame to a temporary file in case of a timeout. This helps preserve progress so that processing can be resumed from the last saved state.

### 4. **Extracting Alignment Scores**
   - **Function:** `get_alignment_score(alignment_text)`
   - **Description:** This function extracts and returns the alignment scores from the alignment text generated by the assistant. It parses the alignment text to identify scores for different categories and compiles them into a dictionary.

### 5. **Replenishing Rate and Token Limits**
   - **Functions:** `replenish_rps()` and `replenish_tps()`
   - **Description:** These functions continuously replenish the rate and token limit semaphores to allow more requests and tokens per second to be processed.

### 6. **Sending and Receiving Data from the Assistant**
   - **Function:** `matchy(formatted_profile, assistant_id, row)`
   - **Description:** This function sends a formatted profile to the assistant for processing, handles rate limits and retries, and returns the alignment text and token usage.

### 7. **Cleaning Text for Processing**
   - **Function:** `clean_text(row)`
   - **Description:** This function formats the text from a row of the DataFrame into a structured string that combines position and applicant profile information. The formatted text is used later for alignment scoring and analysis.

### 8. **Processing a Single Profile**
   - **Function:** `process_batch(row, assistant_id, id_column)`
   - **Description:** This function processes a single profile by cleaning the text, sending it for alignment scoring, and then extracting and returning the relevant alignment scores and token usage.

### 9. **Batch Processing of Profiles**
   - **Function:** `send_and_recive_batch(dataframe, pbar, periodic_save_dir, base_filename, assistant_id, batch, batch_completed, num_in_batch)`
   - **Description:** This function processes a batch of student profiles using concurrent threads, updates the DataFrame with the results, and handles periodic saving. It manages threading for batch processing and updates the progress bar as profiles are processed.

### 10. **Preparing and Managing Batches**
   - **Function:** `prep_batch(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch)`
   - **Description:** Prepares and processes a batch of student profiles by setting up countdown and timeout mechanisms. Manages the timing of batches and handles potential timeouts during batch processing.

### 11. **Batch Handling and Processing**
   - **Function:** `get_batch(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, max_tokens_per_minute, assistant_id, max_rpm)`
   - **Description:** Divides the DataFrame into batches based on token limits and processes each batch. Ensures that the total tokens per minute and requests per minute do not exceed specified limits.

### 12. **Starting the Comparison Process**
   - **Function:** `start_comparisons(SP, pbar, timeout_dir, periodic_save_dir, base_filename, max_tokens_per_minute, assistant_id, max_rpm)`
   - **Description:** Initiates the comparison process by calling the function that processes student profiles in batches.

### 13. **Token Estimation for Profiles**
   - **Functions:** `estimate_tokens(text, model)` and `calculate_profile_tokens(df, model)`
   - **Description:** These functions estimate the number of tokens required for each student profile based on the text length, model, and additional estimated tokens for instructions and output.

### 14. **Processing Time Estimation**
   - **Function:** `estimate_processing_time(len_df, est_total_tokens, max_tokens_per_minute, batch_size)`
   - **Description:** Estimates the total processing time required to compare student profiles based on the number of profiles, estimated tokens, and token limits per minute.

### 15. **Finalizing the Results**
   - **Steps:**
     - Combine the finished output with any previously saved output.
     - Save the final DataFrame to a specified location.
     - Calculate the final cost and total time taken for processing.

### 16. **Global Variables and Resource Management**
   - **Description:** 
     - The script utilizes global variables to track total tokens used, request and token limits, and active threads.
     - Threading is used to manage resource limits and ensure smooth batch processing.

### 17. **Cost and Time Calculation**
   - **Description:** The script calculates the final cost of processing based on the total tokens used and the elapsed time.

### 18. **Output and Display**
   - **Description:** The script displays the estimated processing time, tokens used, and final cost using HTML widgets.


In [ ]:
def load_and_combine_saved_dfs(timeout_dir, base_filename):
    """
    Function Name: load_and_combine_saved_dfs
    
    Purpose/Description:
    Loads all previously saved DataFrames from the specified directory and combines them into a single DataFrame.
    This function is useful for recovering progress from temporary files in case of a timeout or interruption.

    Parameters:
    - timeout_dir (str): Directory path where temporary DataFrame files are saved.
    - base_filename (str): The base filename used for saving the DataFrames.

    Return Value:
    DataFrame: A combined DataFrame containing all the loaded and concatenated data. 
               If no files are found, an empty DataFrame is returned.
    """
    
    # Find all parquet files that match the base filename pattern in the specified directory
    all_files = glob.glob(os.path.join(timeout_dir, f"{base_filename}_*.parquet"))
    
    # Load each file into a DataFrame and store them in a list
    df_list = [pd.read_parquet(file) for file in all_files]
    
    if df_list:
        # Concatenate all DataFrames in the list, remove duplicates, and reset the index
        combined_df = pd.concat(df_list, ignore_index=True).drop_duplicates()
        print(f"Combined {len(df_list)} saved DataFrames.")
    else:
        # If no files are found, return an empty DataFrame
        combined_df = pd.DataFrame()
    
    return combined_df  # Return the combined DataFrame


def restart_processing(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch):
    """
    Function Name: restart_processing
    
    Purpose/Description:
    Restarts the processing of profiles by loading previously saved progress, marking already processed rows, 
    and continuing with the remaining unprocessed profiles. This function is useful for resuming work 
    after a timeout or interruption.

    Parameters:
    - dataframe (DataFrame): The main DataFrame containing all profiles to be processed.
    - pbar (tqdm): The progress bar object to visualize the progress of the comparisons.
    - timeout_dir (str): Directory path where temporary DataFrame files are saved.
    - periodic_save_dir (str): Directory path for saving periodic progress.
    - base_filename (str): The base filename used for saving the DataFrames.
    - assistant_id (str): The ID of the assistant used for generating the alignment scores.
    - batch (DataFrame): The batch of profiles to be processed.
    - num_in_batch (int): The number of profiles in the current batch.

    Return Value:
    None
    """
    
    # Load and combine any saved DataFrames from the previous processing attempts
    combined_df = load_and_combine_saved_dfs(timeout_dir, base_filename)
    
    id_column = 'match-id'  # The unique identifier for each profile-job match
    processed_flag = 'comp_processed'  # Column to flag if the profile has been processed
            
    if not combined_df.empty:
        # Mark rows as processed if they were already included in the combined DataFrame
        dataframe.loc[dataframe[id_column].isin(combined_df[id_column]), processed_flag] = True
        
    # Identify the remaining unprocessed rows
    remaining_df = dataframe[~dataframe[processed_flag]]

    if remaining_df.empty:
        print("No remaining rows to process. All data has been processed.")
    else:
        print(f"{len(combined_df)} row have been processed. {len(remaining_df)} profiles remaining.")
        # Continue processing the remaining profiles
        prep_batch(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch)


def save_temp_dataframe(dataframe, timeout_dir, base_filename):
    """
    Function Name: save_temp_dataframe
    
    Purpose/Description:
    Saves the processed rows of the DataFrame to a temporary file in case of a timeout or interruption. 
    This function helps to preserve progress so that processing can be resumed from the last saved state.

    Parameters:
    - dataframe (DataFrame): The main DataFrame containing all profiles, including those already processed.
    - timeout_dir (str): Directory path where the temporary DataFrame files will be saved.
    - base_filename (str): The base filename used for saving the DataFrames.

    Return Value:
    str: The path to the saved file if rows were saved, otherwise None.
    """
    
    # Filter the DataFrame to only include rows that have been processed
    processed_dataframe = dataframe[dataframe['comp_processed'] == True]
    num_processed = len(processed_dataframe)
    
    if not processed_dataframe.empty:
        # Construct the file path for saving the processed rows
        temp_path = os.path.join(timeout_dir, f"{base_filename}_{num_processed}.parquet")
        # Save the processed rows to a parquet file
        processed_dataframe.to_parquet(temp_path)
        print(f"Saved processed rows to {temp_path} due to timeout.")
        return temp_path  # Return the path to the saved file
    else:
        # If no rows were processed, skip saving
        print("No rows to save; skipping save.")
        return None  # Return None as no file was saved


def get_alignment_score(alignment_text):
    """
    Function Name: get_alignment_score
    
    Purpose/Description:
    Extracts and returns the alignment scores from the alignment text generated by the assistant. 
    This function parses the alignment text to identify the scores for different categories 
    and compiles them into a dictionary.

    Parameters:
    - alignment_text (str): The text containing the alignment score summary.

    Return Value:
    dict: A dictionary containing the extracted scores for each alignment category. 
          If extraction fails, returns a dictionary with an error message and the extracted text.

    Example of Returned Dictionary:
    {
        'Company and Industry Alignment Score': 8,
        'Job Role and Responsibilities vs. Applicant Experience Score': 7,
        'Education, Technical Skills, and Tools': 9,
        'Values, perks, development opportunities, and Company Culture Alignment': 8,
        'Overall Alignment Score': 8
    }
    """
    
    results = {}  # Initialize an empty dictionary to store the results
    
    # Extract the relevant section of the alignment text that contains the scores
    alignment_section = alignment_text.split("Alignment Scores Summary:")[-1].strip()
    
    # Extract the scores using a regular expression to find patterns like "8.0/10"
    scores = re.findall(r'(\d+\.\d+)/10', alignment_section)
    
    # Ensure all five scores are found before assigning them to the dictionary
    if len(scores) == 5:
        results = {
            'Company and Industry Alignment Score': (float(scores[0])),
            'Job Role and Responsibilities vs. Applicant Experience Score': (float(scores[1])),
            'Education, Technical Skills, and Tools Score': (float(scores[2])),
            'Values, perks, development opportunities, and Company Culture Alignment Score': (float(scores[3])),
            'Overall Alignment Score': (float(scores[4]))
        }
    else:
        # If the scores couldn't be extracted correctly, return an error with the extracted text
        results = {'Error': 'Could Not Extract All Scores', 'Extracted Text': alignment_section}
                    
    return results  # Return the dictionary with the scores or the error message


def replenish_rps():
    """
    Function Name: replenish_rps
    
    Purpose/Description:
    Continuously replenishes the rate limit semaphore to allow more requests per second (RPS). 
    This function runs in a loop, waiting for the signal that requests have started and 
    then replenishing the semaphore periodically.
    These values are set in the 'Threads for resource management' region.

    Parameters:
    None

    Return Value:
    None
    """
    while True:
        time.sleep(0.1)  # Short delay to reduce CPU usage
        requests_started.wait()  # Wait until the request process starts
        while True:
            time.sleep(1)  # Wait before replenishing the rate limit
            for _ in range(rps - rate_limit._value):
                rate_limit.release()  # Release the semaphore to allow more requests


def replenish_tps():
    """
    Function Name: replenish_tps
    
    Purpose/Description:
    Continuously replenishes the token limit semaphore to allow more tokens per second (TPS) 
    to be processed. This function runs in a loop, waiting for the signal that requests 
    have started and then replenishing the semaphore periodically. 
    These values are set in the 'Threads for resource management' region. 

    Parameters:
    None

    Return Value:
    None
    """
    while True:
        time.sleep(0.1)  # Short delay to reduce CPU usage
        requests_started.wait()  # Wait until the request process starts
        while True:
            time.sleep(1)  # Wait before replenishing the token limit
            tokens_to_add = tps - token_limit._value  # Calculate how many tokens to add
            if tokens_to_add > 0:
                token_limit.release(tokens_to_add)  # Replenish the semaphore with the required tokens


def matchy(formatted_profile, assistant_id, row):
    """
    Function Name: matchy
    
    Purpose/Description:
    Sends a formatted profile to the assistant for processing, handles rate limits and retries, 
    and returns the alignment text and token usage. This function manages the process of sending 
    requests to the assistant API and handles responses, including managing rate limits.
    
    Note: For most exceptions, a delay and a retry mechanism are implemented. The most common exception will be token and rate limit errors, which will be handled by waiting and retrying the request. 
    Other exceptions will be logged and may trigger a retry or raise an error as needed. Usually a retry works. 

    Parameters:
    - formatted_profile (str): The text of the student profile and job description formatted for the assistant.
    - assistant_id (str): The ID of the assistant used for generating the alignment scores.
    - row (Series): A row from the DataFrame containing information about the job position and applicant, 
      including the number of tokens required.

    Return Value:
    Tuple: A tuple containing:
        - alignment_text (str): The text output from the alignment scoring process.
        - total_tokens_used_in_run (int): The total number of tokens used in the request.
        - total_input_tokens_used_in_run (int): The number of input tokens used.
        - total_output_tokens_used_in_run (int): The number of output tokens used.

    Exceptions/Errors:
    - RateLimitError: Handles rate limit errors and retries the request after waiting.
    - Other Exceptions: Handles unexpected errors and retries or raises them as needed.
    """
    
    global total_tokens_used, total_input_tokens_used, total_output_tokens_used, threads_in_loop  # Global variables to track token usage and active threads
    try:
        # Acquire tokens needed for the request and respect rate limits
        tokens_needed = row['TokenCount_comp']
        token_limit.acquire(tokens_needed)
        rate_limit.acquire()
        
        if not requests_started.is_set():
            requests_started.set()  # Signal that requests have started
            
        while True:
            try:
                # Create a new thread and send the formatted profile as a message
                thread = client.beta.threads.create()
                message = client.beta.threads.messages.create(thread_id=thread.id, content=formatted_profile, role="user")
                run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
                break  # Exit loop if successful
            except Exception as e:
                # Handle rate limit exceeded error (HTTP 429)
                if '429' in str(e):
                    print("Rate limit exceeded. Retrying after a delay.")
                    print("Error:", e)
                    time.sleep(5)  # Wait before retrying
                    continue
                # Handle bad request error (HTTP 400)
                elif '400' in str(e):
                    print(e)
                    continue
                else:
                    raise e  # Raise other exceptions
                
        with threads_lock:
            threads_in_loop += 1  # Increment active threads count
        
        while True:
            time.sleep(2)
            try:
                # Check the status of the run
                run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                if run_status.status == "completed":
                        
                    # Update the total tokens used globally
                    total_input_tokens_used_in_run = run_status.usage.prompt_tokens
                    total_input_tokens_used += total_input_tokens_used_in_run
                    
                    total_output_tokens_used_in_run = run_status.usage.completion_tokens
                    total_output_tokens_used += total_output_tokens_used_in_run
                    
                    total_tokens_used_in_run = run_status.usage.total_tokens
                    total_tokens_used += total_tokens_used_in_run
                    
                    break  # Exit loop when processing is complete

                elif run_status.status == "failed":
                    # Handle rate limit exceeded within the run
                    if run_status.last_error.code == 'rate_limit_exceeded':
                        match1 = re.search(r'in (\d+\.\d+)s', run_status.last_error.message)
                        match2 = re.search(r'in (\d+)ms', run_status.last_error.message)
                        if match1:
                            wait_time = float(match1.group(1)) + 1
                            print(run_status.last_error.message)
                            print('#ERROR3#: Rate limit exceeded. Waiting for', wait_time, 'seconds.')
                            with threads_lock:
                                threads_in_loop -= 1
                            time.sleep(wait_time)
                            return matchy(formatted_profile, assistant_id, row)
                        elif match2:
                            wait_time = float(match2.group(1)) / 1000.0 + 1  # Convert milliseconds to seconds
                            print(run_status.last_error.message)
                            print('#ERROR3#: Rate limit exceeded. Waiting for', wait_time, 'seconds.')
                            with threads_lock:
                                threads_in_loop -= 1
                            time.sleep(wait_time)
                            return matchy(formatted_profile, assistant_id, row)
                    # Handle a generic error message
                    elif 'Sorry, something went wrong' in str(run_status.last_error):
                        time.sleep(2)
                        with threads_lock:
                            threads_in_loop -= 1
                        return matchy(formatted_profile, assistant_id, row)
                    else:
                        print("Run failed. Trying again", run_status.last_error)
                        with threads_lock:
                            threads_in_loop -= 1
                        return matchy(formatted_profile, assistant_id, row)   
            except RateLimitError as e:
                print('#ERROR2#: Rate limit exceeded.')
                time.sleep(2)
                with threads_lock:
                    threads_in_loop -= 1
                return matchy(formatted_profile, assistant_id, row)   
            except Exception as e:
                if 'rate_limit_exceeded' in str(e):
                    print('#ERROR2#: Rate limit exceeded. waiting and retrying.')
                    with threads_lock:
                        threads_in_loop -= 1
                    time.sleep(10)
                    return matchy(formatted_profile, assistant_id, row)
                else:
                    print("Error:", e)
                    raise e  # Raise other exceptions
                
        with threads_lock:
            threads_in_loop -= 1  # Decrement active threads count

        # Retrieve and return the result message from the assistant
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        for message in reversed(messages.data):
            role = message.role  
            for content in message.content:
                if content.type == 'text' and role == 'assistant':
                    return content.text.value, total_tokens_used_in_run, total_input_tokens_used_in_run, total_output_tokens_used_in_run

    except RateLimitError as e:
        print('#ERROR1#: Rate limit exceeded.')
        time.sleep(3)
    except Exception as e:
        if 'rate_limit_exceeded' in str(e):
            time.sleep(3)
            return matchy(formatted_profile, assistant_id, row)
        else:
            raise e  # Raise other exceptions if not related to rate limit


def clean_text(row):
    """
    Function Name: clean_text
    
    Purpose/Description:
    Formats the text from a row of the DataFrame into a structured string that combines 
    position and applicant profile information. This formatted text is used later for 
    alignment scoring and analysis. This is the exact format that gpt-4o will recive the information in.

    Parameters:
    - row (Series): A row from the DataFrame containing information about a job position 
      and the corresponding applicant.

    Return Value:
    str: A formatted string containing the combined information of the job position and applicant profile.
    """

    # Create a formatted string containing job position details and applicant information
    formatted_text = f"""
    
    -----------------------------------------------
    Position Profile 
    -----------------------------------------------

    Company Name: {row['pos_Company']}
    Position Title: {row['pos_Name']}

    Position Description:
    -----------------------------------------------

    {row['pos_Job_desc_text']}

    Other Skills/Requirements/Preferences Summary
    -----------------------------------------------

    Other requirements/preferences: 
    {row['pos_Other requirements/preferences']}

    Other Skills:
    {row['pos_Other Skills']}

    Position Summary:
    {row['pos_External Position Summary']}
    
    Position Skill Summary:
    {row['pos_Position_skill_summary']}
    
    ----------------------------------------
    Applicant Profile:
    ----------------------------------------

    Applicant Name: {row['stu_Legal Name']}

    Applicant Resume: 
    ----------------------------------------
    {row['stu_Resume_text']}

    Applicant Essays:
    ----------------------------------------

    Essay - Dream Companies
    {row['stu_Essay - Dream Companies']}

    Essay - Experience in field of Study
    {row['stu_Essay - Experience in field of Study']}

    Essay - Influence on interest in the field
    {row['stu_Essay - Influence on interest in the field']}

    Essay - Internship Career Goals
    {row['stu_Essay - Internship Career Goals']}

    Essay - Overall Career Goals
    {row['stu_Essay - Overall Career Goals']}

    Applicant Skill/Education Summary
    ----------------------------------------

    {row['stu_Position_skill_summary']}
    
    """
    
    return formatted_text  # Return the formatted string


def process_batch(row, assistant_id, id_column):
    """
    Function Name: process_batch
    
    Purpose/Description:
    Processes a single profile by cleaning the text, sending it for alignment scoring, 
    and then extracting and returning the relevant alignment scores and token usage.

    Parameters:
    - row (Series): A row from the DataFrame containing information about a job position 
      and the corresponding applicant.
    - assistant_id (str): The ID of the assistant used for generating the alignment scores.
    - id_column (str): The name of the column containing the unique identifier for each 
      profile-job match.

    Return Value:
    Tuple: A tuple containing:
        - profile_id (str): The unique identifier of the profile-job match.
        - alignment_text (str): The text output from the alignment scoring process.
        - alignment_scores (dict): A dictionary containing the alignment scores.
        - actual_tokens_used (int): The number of tokens used during the alignment process.
        - actual_input_tokens_used (int): The number of input tokens used.
        - actual_output_tokens_used (int): The number of output tokens used.
    """
    
    # Clean the text from the row to prepare it for alignment scoring
    cleaned_text = clean_text(row)
    
    # Send the cleaned text to the assistant for alignment scoring and receive the results
    alignment_text, actual_tokens_used, actual_input_tokens_used, actual_output_tokens_used = matchy(cleaned_text, assistant_id, row)

    # Extract alignment scores from the returned alignment text
    alignment_scores = get_alignment_score(alignment_text)

    # Return the profile's unique ID, alignment text, scores, and token usage information
    return row[id_column], alignment_text, alignment_scores, actual_tokens_used, actual_input_tokens_used, actual_output_tokens_used


def send_and_recive_batch(dataframe, pbar, periodic_save_dir, base_filename, assistant_id, batch, batch_completed, num_in_batch):
    """
    Function Name: send_and_recive_batch
    
    Purpose/Description:
    Processes a batch of student profiles using concurrent threads, updates the DataFrame with the results, 
    and handles periodic saving. The function manages the threading for batch processing and updates the 
    progress bar as profiles are processed.

    Parameters:
    - dataframe (DataFrame): The DataFrame containing student profiles and job descriptions.
    - pbar (tqdm): The progress bar object to visualize the progress of the comparisons.
    - periodic_save_dir (str): Directory path for saving periodic progress.
    - base_filename (str): The base filename used for saving files.
    - assistant_id (str): The ID of the assistant used for generating the alignment scores.
    - batch (DataFrame): The batch of student profiles to be processed.
    - batch_completed (threading.Event): Event to signal the completion of batch processing.
    - num_in_batch (int): The number of profiles in the current batch.

    Return Value:
    None
    """
    
    global total_comparisons, num_comparissons  # Use global variables to track comparisons

    id_column = 'match-id'  # Unique identifier for each profile-job match
    ''' Note on the number of workers used in the ThreadPoolExecutor:
    This is a very I/O bound task, so the number of workers can be set much higher than the number of CPU cores. 
    We keep the number of workers equal to the number of profiles in the batch to process them concurrently.
    This also helps in utilizing the available tokens more efficiently and avoiding rate limits. 
    If rate limits become an issue, the number of workers can be reduced.
    '''
    workers = num_in_batch  # Number of worker threads to use, based on batch size.

    # #region as_completed code
    with ThreadPoolExecutor(max_workers=workers) as executor:
        # Submit each row in the batch for processing
        future_to_profile = {
            executor.submit(process_batch, row, assistant_id, id_column): row
            for idx, row in batch.iterrows()
        }

        # Process each completed future as it finishes
        for future in as_completed(future_to_profile):
            try:
                # Retrieve the result from the future
                result = future.result()
                profile_id, alignment_text, alignment_scores, actual_tokens_used, actual_input_tokens_used, actual_output_tokens_used = result
                
                # #region updating the DataFrame with results
                
                # Mark the profile as processed
                dataframe.loc[dataframe[id_column] == profile_id, 'comp_processed'] = True
                
                # Extract and store the alignment scores from the result
                overall_alignment_score = alignment_scores.get('Overall Alignment Score')
                company_and_industry_alignment_score = alignment_scores.get('Company and Industry Alignment Score')
                job_role_and_responsibilities_score = alignment_scores.get('Job Role and Responsibilities vs. Applicant Experience Score')
                education_technical_skills_and_tools_score = alignment_scores.get('Education, Technical Skills, and Tools Score')
                values_perks_development_culture_score = alignment_scores.get('Values, perks, development opportunities, and Company Culture Alignment Score')
                
                # Update the DataFrame with the alignment text and scores
                dataframe.loc[dataframe[id_column] == profile_id, 'Alignment Text'] = alignment_text 
                dataframe.loc[dataframe[id_column] == profile_id, 'Overall Alignment Score'] = overall_alignment_score
                dataframe.loc[dataframe[id_column] == profile_id, 'Company and Industry Alignment Score'] = company_and_industry_alignment_score
                dataframe.loc[dataframe[id_column] == profile_id, 'Job Role and Responsibilities vs. Applicant Experience Score'] = job_role_and_responsibilities_score
                dataframe.loc[dataframe[id_column] == profile_id, 'Education, Technical Skills, and Tools Score'] = education_technical_skills_and_tools_score
                dataframe.loc[dataframe[id_column] == profile_id, 'Values, perks, development opportunities, and Company Culture Alignment Score'] = values_perks_development_culture_score
                
                # Update the DataFrame with the actual tokens used in processing
                dataframe.loc[dataframe[id_column] == profile_id, 'Comp Actual Tokens Used'] = actual_tokens_used
                dataframe.loc[dataframe[id_column] == profile_id, 'Comp Actual Input Tokens Used'] = actual_input_tokens_used
                dataframe.loc[dataframe[id_column] == profile_id, 'Comp Actual Output Tokens Used'] = actual_output_tokens_used     
                
                # Increment the comparison counter and periodically save the DataFrame
                num_comparissons += 1
                if num_comparissons % 50 == 0:
                    path = f"{periodic_save_dir}/{base_filename}_{num_comparissons}.parquet"
                    dataframe.to_parquet(path)
                pbar.update(1)  # Update the progress bar
                
            except Exception as e:
                # Print any errors that occur during processing
                print("Error processing profile:", e)
                continue
    
        # Signal that the batch processing is complete
        batch_completed.set()


def prep_batch(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch):
    """
    Function Name: prep_batch
    
    Purpose/Description:
    Prepares and processes a batch of student profiles by setting up countdown and timeout mechanisms. 
    Manages the timing of batches and handles potential timeouts during batch processing.

    Parameters:
    - dataframe (DataFrame): The DataFrame containing student profiles and job descriptions.
    - pbar (tqdm): The progress bar object to visualize the progress of the comparisons.
    - timeout_dir (str): Directory path where temporary files will be saved in case of a timeout.
    - periodic_save_dir (str): Directory path for saving periodic progress.
    - base_filename (str): The base filename used for saving files.
    - assistant_id (str): The ID of the assistant used for generating the alignment scores.
    - batch (DataFrame): The batch of student profiles to be processed.
    - num_in_batch (int): The number of profiles in the current batch.

    Return Value:
    None
    """

    # Event to signal when the countdown timer has finished
    countdown_finished = threading.Event()

    def countdown_timer(seconds):
        """Runs a countdown timer for 60 seconds. Required for keeping token usage per minute under the limits."""
        with tqdm(total=seconds, desc=f"Processing {num_in_batch} rows. Time Till Next Batch:", 
                  bar_format="{l_bar}{bar}| {remaining} seconds", leave=False) as pbar:
            for i in range(seconds):
                time.sleep(1)
                pbar.update(1)
        countdown_finished.set()  # Signal that the countdown has finished

    def timeout_handler():
        """Handles the situation where processing takes too long and triggers a timeout."""
        if countdown_finished.is_set():
            save_temp_dataframe(dataframe, timeout_dir, base_filename)  # Save progress if the countdown is complete
            print("Timeout occurred, saving current state and restarting process.")
            restart_processing(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch)
        else:
            print("Timeout occurred, but batch has already started processing. Ignoring timeout.")

    # Start the countdown timer in a separate thread
    countdown_thread = threading.Thread(target=countdown_timer, args=(60,))
    countdown_thread.start()

    # Set up a timeout handler to trigger after 300 seconds if the batch hasn't completed
    timeout_thread = threading.Timer(300, timeout_handler)
    timeout_thread.start()
    
    try:
        # Process the batch and wait for completion
        send_and_recive_batch(dataframe, pbar, periodic_save_dir, base_filename, assistant_id, batch, batch_completed, num_in_batch)
        countdown_finished.wait()  # Wait until the countdown finishes
        batch_completed.wait()  # Wait until the batch processing completes
    finally:
        # Clean up threads and reset events
        countdown_thread.join()  # Ensure the countdown thread finishes
        timeout_thread.cancel()  # Cancel the timeout thread if it hasn't triggered
        batch_completed.clear()  # Clear the batch completed event


def get_batch(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, max_tokens_per_minute, assistant_id, max_rpm):
    """
    Function Name: get_batch
    
    Purpose/Description:
    Divides the DataFrame into batches based on token limits and processes each batch. 
    Ensures that the total tokens per minute and requests per minute do not exceed specified limits.

    Parameters:
    - dataframe (DataFrame): The DataFrame containing student profiles and job descriptions.
    - pbar (tqdm): The progress bar object to visualize the progress of the comparisons.
    - timeout_dir (str): Directory path where temporary files will be saved in case of a timeout.
    - periodic_save_dir (str): Directory path for saving periodic progress.
    - base_filename (str): The base filename used for saving files.
    - max_tokens_per_minute (int): The maximum number of tokens that can be processed per minute.
    - assistant_id (str): The ID of the assistant used for generating the alignment scores.
    - max_rpm (int): The maximum number of requests per minute allowed.

    Return Value:
    None
    """
    
    id_column = dataframe['match-id']  # Identifier for each profile-job match
    batch = pd.DataFrame(columns=dataframe.columns)  # Initialize an empty DataFrame for the batch
    batch_token_count = 0  # Counter for the total tokens in the current batch
    num_in_batch = 0  # Counter for the number of profiles in the current batch
    
    # Iterate over each row in the DataFrame to create batches
    for idx, row in dataframe.iterrows():
        if row['comp_processed']:
            continue  # Skip rows that have already been processed
        
        current_token_count = int(row['TokenCount_comp'])  # Get the token count for the current row

        # Check if adding the current row would exceed the token or request limits
        if batch_token_count + current_token_count > max_tokens_per_minute or num_in_batch == max_rpm:
            # Process the current batch
            prep_batch(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch)

            # Reset the batch and token counter for the next batch
            batch = pd.DataFrame(columns=dataframe.columns)
            batch_token_count = 0
            num_in_batch = 0
        
        # Add the current row to the batch
        if batch.empty:
            batch = pd.DataFrame([row])
        else:
            batch = pd.concat([batch, pd.DataFrame([row])], ignore_index=True)
            
        batch_token_count += current_token_count  # Update the token count for the batch
        num_in_batch += 1  # Increment the batch size
    
    # Process any remaining rows in the final batch
    if not batch.empty:
        prep_batch(dataframe, pbar, timeout_dir, periodic_save_dir, base_filename, assistant_id, batch, num_in_batch)


def start_comparisons(SP, pbar, timeout_dir, periodic_save_dir, base_filename, max_tokens_per_minute, assistant_id, max_rpm):
    """
    Function Name: start_comparisons
    
    Purpose/Description:
    Initiates the comparison process by calling the function that processes student profiles in batches.

    Parameters:
    - SP (DataFrame): The DataFrame containing student profiles and job descriptions.
    - pbar (tqdm): The progress bar object to visualize the progress of the comparisons.
    - timeout_dir (str): Directory path where temporary files will be saved in case of a timeout.
    - periodic_save_dir (str): Directory path for saving periodic progress.
    - base_filename (str): The base filename used for saving files.
    - max_tokens_per_minute (int): The maximum number of tokens that can be processed per minute.
    - assistant_id (str): The ID of the assistant used for generating the alignment scores.
    - max_rpm (int): The maximum number of requests per minute allowed.

    Return Value:
    None
    """
    get_batch(SP, pbar, timeout_dir, periodic_save_dir, base_filename, max_tokens_per_minute, assistant_id, max_rpm)


def estimate_tokens(text, model):
    """
    Function Name: estimate_tokens
    
    Purpose/Description:
    Estimates the number of tokens in a given text based on the encoding model used.

    Parameters:
    - text (str): The input text for which token estimation is required.
    - model (str): The model used for encoding the text (e.g., 'gpt-4o-mini').

    Return Value:
    int: The number of tokens estimated in the input text.
    """
    encoding = tiktoken.encoding_for_model(model)  # Get the encoding for the specified model
    tokens = encoding.encode(text)  # Encode the text to determine the number of tokens
    return len(tokens)  # Return the number of tokens


def calculate_profile_tokens(df, model):
    """
    Function Name: calculate_profile_tokens
    
    Purpose/Description:
    Calculates the estimated number of tokens required for each student profile by 
    considering the text length, model, and additional estimated tokens for instructions and output.

    Parameters:
    - df (DataFrame): The DataFrame containing student profiles and job descriptions.
    - model (str): The model used for token estimation.

    Return Value:
    DataFrame: The updated DataFrame with estimated token counts for each profile.
    """
    
    id_column = 'match-id'  # Unique identifier for each profile-job match
    
    # Estimate token counts for instructions and average output
    instructions_token_est = 1000  # Estimated tokens for instructions (Because each API call is on its own thread, instruction tokens are included in every call.)
    avg_token_output =  700  # Estimated tokens for the output
    
    for idx, row in df.iterrows():
        formatted_text = clean_text(row)  # Clean and format the text for the profile
        token_count = estimate_tokens(formatted_text, model)  # Estimate the number of tokens for the text
        
        # Update the DataFrame with the estimated token counts
        df.loc[df[id_column] == row[id_column], 'TokenCount_comp'] = token_count + instructions_token_est + avg_token_output
        df.loc[df[id_column] == row[id_column], 'TokenInputCount_comp'] = token_count + instructions_token_est
        df.loc[df[id_column] == row[id_column], 'TokenOutputCount_comp'] = avg_token_output
        
    return df  # Return the updated DataFrame


def estimate_processing_time(len_df, est_total_tokens, max_tokens_per_minute, batch_size):
    """
    Function Name: estimate_processing_time
    
    Purpose/Description:
    Estimates the total processing time required to compare student profiles based on the number of profiles,
    estimated tokens, and token limits per minute.

    Parameters:
    - len_df (int): The number of rows in the DataFrame (i.e., the number of profiles to process).
    - est_total_tokens (int): The estimated total number of tokens required for processing all profiles.
    - max_tokens_per_minute (int): The maximum number of tokens that can be processed per minute.
    - batch_size (int): The number of profiles processed in each batch.

    Return Value:
    Tuple[int, int]: A tuple containing the estimated hours and minutes required for processing.
    """
    # Calculate the number of batches required to process all profiles
    num_batches = math.ceil(len_df / batch_size)
    
    # Determine the time required based on whether token limit or request limit is the bottleneck
    if est_total_tokens / num_batches <= max_tokens_per_minute:
        minutes_required = num_batches  # Limited by requests per minute
    else:
        minutes_required = est_total_tokens / max_tokens_per_minute  # Limited by tokens per minute
    
    minutes_required = math.ceil(minutes_required)  # Round up to the nearest minute
    
    # Convert total minutes to hours and minutes
    est_hours = minutes_required // 60  # Calculate hours
    est_minutes = minutes_required % 60  # Calculate remaining minutes
    
    return est_hours, est_minutes  # Return estimated processing time in hours and minutes


# Define the model to use for generating alignment scores
'''
Note: ALWAYS USE THE 'gpt-4o-mini' MODEL IF YOU ARE ONLY TESTING THE CODE. 
It's 50x cheaper than the 'gpt-4o' model, and if you are running/modifying the code, you will likely need to run it over and over, and the output matters less. 
Switch to 'gpt-4o' if the primary focus is the output.
'''
model = 'gpt-4o-mini'  # Set the model to 'gpt-4o-mini' for testing or 'gpt-4o' for final runs.
API_tier = 3  # Set the API tier based on your OpenAI account level.

# Define the maximum number of requests per minute to avoid rate limiting
max_rpm = 150

# Start a timer to measure the total time taken for the entire process
start_time = time.time()

# Create a new assistant using OpenAI's API with the specified model and instructions
matchy_assistent = client.beta.assistants.create(
    name="Student Profile Technical Skill and Education Extract-O-Bot",  # Name of the assistant
    instructions=instruction,  # Instructions for the assistant on how to perform the task
    model=model,  # Model to be used for the task
    temperature=1,  # Controls the creativity of the model's responses (1 means more creative)
    top_p=1  # Controls the diversity of the model's responses (1 means maximum diversity)
)


# Initialize columns in the DataFrame 'SP' to store alignment scores and text outputs
#region alignment text and scores columns
SP['Alignment Text'] = ''  # Column to store the alignment text generated by the assistant
SP['Overall Alignment Score'] = 0.0  # Column to store the overall alignment score
SP['Company and Industry Alignment Score'] = 0.0  # Column to store the company and industry alignment score
SP['Job Role and Responsibilities vs. Applicant Experience Score'] = 0.0  # Column to store job role alignment score
SP['Education, Technical Skills, and Tools Score'] = 0.0  # Column to store education and skills alignment score
SP['Values, perks, development opportunities, and Company Culture Alignment Score'] = 0.0  # Column to store values and culture alignment score
SP['match-id'] = SP['pos_(Do Not Modify) Job Posting'] + '_' + SP['stu_(Do Not Modify) Application']  # Unique identifier for each profile-job match
SP['comp_processed'] = False  # Flag to indicate whether the profile has been processed or not
#endregion

# Estimate the maximum number of tokens that can be processed per minute based on the model used
#region token and time estimation

#region tokens per minute
''' 
Note: The maximum number of tokens per minute varies depending on your OpenAI API tier and the model used.
Info on tokens per minute for different models and API tiers can be found here: https://platform.openai.com/docs/guides/rate-limits
Info on your specific API tier and rate limits can be found in the 'Your Profile' section under 'Limits' on the OpenAI platform.
If you are using tier 1, doing a large number of comparisons will be slooowww (2+ hours for 1000 comparisons). You can only process about 4-5 rows a minute. 
Once you are tier 2 or higher, you can process 150 rows a minute and 1000 rows in 7 minutes.
'''

# Determine max tokens per minute based on the model and API tier
if model == 'gpt-4o-mini':
    if API_tier == 1:
        max_tokens_per_minute = 200000  # Tier 1 limit for 'gpt-4o-mini'
    elif API_tier == 2:
        max_tokens_per_minute = 2000000  # Tier 2 limit for 'gpt-4o-mini'
    elif API_tier == 3:
        max_tokens_per_minute = 4000000  # Tier 3 limit for 'gpt-4o-mini'
    elif API_tier == 4:
        max_tokens_per_minute = 10000000  # Tier 4 limit for 'gpt-4o-mini'
    elif API_tier == 5:
        max_tokens_per_minute = 150000000  # Tier 5 limit for 'gpt-4o-mini'
    else:
        raise ValueError("Unsupported API tier for 'gpt-4o-mini'")

elif model == 'gpt-4o':
    if API_tier == 1:
        max_tokens_per_minute = 25000  # Tier 1 limit for 'gpt-4o'
    elif API_tier == 2:
        max_tokens_per_minute = 450000  # Tier 2 limit for 'gpt-4o'
    elif API_tier == 3:
        max_tokens_per_minute = 800000  # Tier 3 limit for 'gpt-4o'
    elif API_tier == 4:
        max_tokens_per_minute = 2000000  # Tier 4 limit for 'gpt-4o'
    elif API_tier == 5:
        max_tokens_per_minute = 30000000  # Tier 5 limit for 'gpt-4o'
    else:
        raise ValueError("Unsupported API tier for 'gpt-4o'")

else:
    print('I only made code for gpt-4o and gpt-4o-mini. If you want to use a different model, you will have to add the token limits yourself.')
    token_limit_input = input('Enter a token limit per minute for the model you are using: ')
    
    # Convert the input to an integer and set it as the max_tokens_per_minute
    try:
        max_tokens_per_minute = int(token_limit_input.replace(",", "").strip())  # Remove commas if present and convert to integer
    except ValueError:
        raise ValueError("Invalid input. Please enter a valid integer for the token limit.")

#endregion

# Initialize columns in the DataFrame 'SP' to track the actual number of tokens used
#region Actual tokens used columns
SP['Actual Tokens Used'] = 0  # Total number of tokens used
SP['Actual Input Tokens Used'] = 0  # Number of input tokens used
SP['Actual Output Tokens Used'] = 0  # Number of output tokens used
#endregion

# Initialize columns in the DataFrame 'SP' to estimate the number of tokens needed
#region token count columns
SP['TokenCount_comp'] = 0  # Estimated total number of tokens for each comparison
SP['TokenInputCount_comp'] = 0  # Estimated number of input tokens for each comparison
SP['TokenOutputCount_comp'] = 0  # Estimated number of output tokens for each comparison
#endregion

# Calculate the estimated number of tokens required for each profile-job comparison
SP = calculate_profile_tokens(SP, model)

# Estimate the total token usage and the cost associated with processing the profiles
#region tokens use and price estimation

# Define the relevant columns for token calculations
token_cols = ['TokenCount_comp',  # Total token count for comparison
              'TokenInputCount_comp',  # Input token count for comparison
              'TokenOutputCount_comp',  # Output token count for comparison
              ]

# Calculate the total tokens required for all profiles
token_subset = SP[token_cols]
est_total_tokens = token_subset['TokenCount_comp'].sum()  # Sum of total tokens for all comparisons
est_total_input_tokens = token_subset['TokenInputCount_comp'].sum()  # Sum of input tokens for all comparisons
est_total_output_tokens = token_subset['TokenOutputCount_comp'].sum()  # Sum of output tokens for all comparisons

# Compute the maximum token value for a single comparison
max_token_value = SP['TokenCount_comp'].max()

#region cost per token
# Define the cost per token based on the model used
if model == 'gpt-4o-mini':
    price_per_input_token = 0.00000015  # Cost per input token for 'gpt-4o-mini'
    price_per_output_token = 0.00000030  # Cost per output token for 'gpt-4o-mini'
elif model == 'gpt-4o':
    price_per_input_token = 0.000005  # Cost per input token for 'gpt-4o'
    price_per_output_token = 0.000015  # Cost per output token for 'gpt-4o'

# Estimate the total price for processing all profiles
est_total_price = (est_total_input_tokens * price_per_input_token) + (est_total_output_tokens * price_per_output_token)
#endregion

#endregion

# Estimate the total time required to process all profiles based on the token usage and batch size
#region time estimation
len_df = len(SP)  # Number of rows in the DataFrame (total profiles to be processed)
est_hours, est_minutes = estimate_processing_time(len_df, est_total_tokens, max_tokens_per_minute, max_rpm)  # Estimated time in hours and minutes
#endregion

#region Display the token and time estimation results

# Create HTML widgets to display the estimated processing details to the user
total_comparisons_label = widgets.HTML(value=f"<b>Total number of profiles to compare:</b> {len(SP)}")  # Display the total number of profiles
est_tokens_label = widgets.HTML(value=f"<b>Estimate for total number of tokens required:</b> {est_total_tokens}")  # Display estimated total tokens needed
est_price_label = widgets.HTML(value=f"<b>Estimate for total price for processing:</b> ${est_total_price:.2f}")  # Display estimated total price
est_time_label = widgets.HTML(value=f"<b>Estimated time required:</b> {est_hours} hours and {est_minutes} minutes")  # Display estimated time required

# Display all the estimation results to the user
display(total_comparisons_label, est_tokens_label, est_price_label, est_time_label)
#endregion

#endregion



#region Global variables

# Initialize global variables to track total tokens used in the process
total_tokens_used = 0  # Track total tokens used for the entire process
total_input_tokens_used = 0  # Track total input tokens used
total_output_tokens_used = 0  # Track total output tokens used
#endregion


#region Threads for resource management

# Set the rate and token limits for processing
''' 
Note: You will hit rate limit errors if you send too many API requests at the same time, so you have to lower the number of batches you send a second. 
This is handled by using semaphores in functions about matchy. Adjust these to higher per second values if you are using a higher tier. 
They are set to the lowest possible values for tier 1.
'''
rps = 5  # Requests per second
tps = 20000 # Tokens per second (Don't lower this beyond 10k or else some rows with a higher token count will not be processed)

# Create a threading lock to manage access to shared resources
threads_lock = threading.Lock()

# Create semaphores to control the rate and token limits
rate_limit = threading.Semaphore(rps)  # Semaphore for rate limiting
token_limit = threading.Semaphore(tps)  # Semaphore for token limiting

# Start threads to replenish the rate and token limits continuously
threading.Thread(target=replenish_rps, daemon=True).start()  # Thread to replenish requests per second
threading.Thread(target=replenish_tps, daemon=True).start()  # Thread to replenish tokens per second

# Create threading events to synchronize batch completion and request initiation
batch_completed = threading.Event()  # Event to signal batch completion
requests_started = threading.Event()  # Event to signal the start of requests
#endregion


#region thread monitoring

# Initialize a counter to track the number of threads in the loop
threads_in_loop = 0

# Optional code to monitor the number of active threads waiting for API call completions, currently commented out
# def monitor_loop_threads():
#     requests_started.wait()
#     while True:
#         with threads_lock:
#             print(f"Threads in loop: {threads_in_loop}")
#         time.sleep(5)  # Print the number of threads in the loop every 5 seconds

# # Start the loop monitoring in a background thread
# threading.Thread(target=monitor_loop_threads, daemon=True).start()

#endregion


#region define path for saving progress

# Define paths for saving progress and final results
''' 
Note: Sometimes an API request can get stuck, so I've added a reset process. You will need directories for periodic saves, timeouts, and the final save. 
'''
timeout_dir = f'{path_to_project}/data/matchy_saves/timeout_dir'  # Directory for saving progress in case of timeout
periodic_save_dir = f'{path_to_project}/data/matchy_saves/periodic_save_dir'  # Directory for periodic saves
final_save_path = f'{path_to_project}/data/matchy_saves/final_save_dir/matchy_final_save.parquet'  # Final save path for processed data
#endregion


#region counter for progress bar

# Initialize counters for the progress bar
total_comparisons = len(SP)  # Total number of profiles to compare
num_comparissons = 0  # Counter to track the number of comparisons made
#endregion



# Display the progress bar and start the comparison process
with tqdm(total=total_comparisons, desc="Comparing Profiles") as pbar:
    start_comparisons(SP, pbar, timeout_dir, periodic_save_dir, 'SP_extraction_save', max_tokens_per_minute, matchy_assistent.id, max_rpm)


#region final save

# Combine the finished output with any previously saved output
combined_df = load_and_combine_saved_dfs(timeout_dir, 'SP_matchy_save')

# Save the final DataFrame if no previous saves exist
if combined_df.empty:
    SP.to_parquet(final_save_path)   
else: 
    # Combine the new data with previously saved data and save the final DataFrame
    final_df = pd.concat([SP, combined_df], ignore_index=True).drop_duplicates()
    final_df.to_parquet(final_save_path)
    
#endregion
    

#region Final token and time calculation

# Calculate the final cost and total time taken for processing
total_price = (total_input_tokens_used * price_per_input_token) + (total_output_tokens_used * price_per_output_token)  # Calculate total price for processing
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time  # Calculate the total elapsed time

# Convert the elapsed time into hours, minutes, and seconds for display
elapsed_hours = int(elapsed_time // 3600)
elapsed_minutes = int((elapsed_time % 3600) // 60)
elapsed_seconds = int(elapsed_time % 60)
formatted_time = f"{elapsed_hours}h {elapsed_minutes}m {elapsed_seconds}s"  # Format the elapsed time

# Create HTML widgets to display the final processing results
process_complete_label = widgets.HTML(value=f"<b>Processing complete. Compared {total_comparisons} profiles. Final DataFrame saved to:</b> {final_save_path}")
tokens_label = widgets.HTML(value=f"<b>Total tokens used:</b> {total_tokens_used}")  # Display total tokens used
price_label = widgets.HTML(value=f"<b>Total price for processing:</b> ${total_price:.2f}")  # Display total price for processing
time_label = widgets.HTML(value=f"<b>Total time used:</b> {formatted_time}")  # Display total time taken

#endregion


# Display the final results to the user
display(tokens_label, price_label, time_label)


### Final Results

We now have alignment scores for each student-job pair in the DataFrame `SP`. These scores will be utilized in the next pipeline to generate a final ranking of students for each job position. Below, we can review some of the results produced by the Matchy-9000 process.

In [8]:
# This is a path to a df containing alignment scores for each student-job pair from 2024 using gpt-4o-mini. If you generated your own, you can comment this out and use that instead.
SP_path = f'{path_to_project}/data/SP_table/SP5_post_matchy.parquet'
SP = pd.read_parquet(SP_path) 

In [ ]:
# Below is a list of Companies from 2024. If you'd like to see the matches for a specific company, copy and paste the name in the 'Company_Name' variable below.

for company in SP['pos_Company'].unique():
    print(company)

In [ ]:

cols = ''' 
stu_Legal Name
pos_Name
pos_Company
Alignment Text
Overall Alignment Score
Company and Industry Alignment Score
Job Role and Responsibilities vs. Applicant Experience Score
Education, Technical Skills, and Tools Score
Values, perks, development opportunities, and Company Culture Alignment Score
'''

cols = as_list(cols)

alignment_scores = SP[cols]

Company_Name = 'Media Group'

alignment_scores = (alignment_scores[alignment_scores['pos_Company'] == Company_Name])

# sort alignment scores by overall alignment score
alignment_scores = alignment_scores.sort_values(by='Overall Alignment Score', ascending=False)

pretty_print(alignment_scores)